<a href="https://colab.research.google.com/github/keran-w/Projects-in-ML-and-AI/blob/main/ProjML%26AI_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# connect to google drive
from google.colab import drive
from IPython.display import clear_output
drive.mount('/content/drive', force_remount=True)
clear_output()

import warnings
warnings.filterwarnings("ignore")

In [3]:
# import dataset from kaggle
# dataset link https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
import os
if not os.path.isfile('~/.kaggle/kaggle.json'):
    os.system('mkdir ~/.kaggle')
    os.system('cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json')
!kaggle datasets download -d uciml/pima-indians-diabetes-database
!unzip pima-indians-diabetes-database.zip -d .

  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 8.13MB/s]
Archive:  pima-indians-diabetes-database.zip
  inflating: ./diabetes.csv          


In [4]:
# read file using pandas
import pandas as pd
data = pd.read_csv('diabetes.csv')
data.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#### Task 1 (30 points): 

Implement a Decision Tree Classifier for your classification problem. You
may use a built-in package to implement your classifier. Try modifying one or more of the input
parameters and describe what changes you notice in your results. Clearly describe how these
factors are affecting your output.

#### Task 2 (30 points): 

From the Bagging and Boosting ensemble methods pick any one algorithm
from each category. Implement both the algorithms using the same data. Use k-fold cross
validation to find the effectiveness of both the models. Comment on the difference/similarity of
the results.

#### Task 3 (40 points): 

Compare the effectiveness of the three models implemented above. Clearly
describe the metric you are using for comparison. Describe (with examples) Why is this
metric(metrics) suited/appropriate for the problem at hand? How would a choice of a different
metric impact your results? Can you demonstrate that?